In [197]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install lxml  # Optional, but recommended for better HTML parsing
!pip install html5lib  # Optional, another HTML parsing library
!pip install selenium webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#'


Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '#'


Defaulting to user installation because normal site-packages is not writeable


In [198]:
import pandas as pd
import time
import csv
import logging
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [199]:
# Configure logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('straits_times_scraper_debug.log'),
        logging.StreamHandler()
    ]
)

In [200]:
def setup_webdriver():
    """
    Set up Selenium WebDriver with Chrome and comprehensive error handling
    
    Returns:
        webdriver: Configured Chrome WebDriver
    """
    try:
        # Chrome options
        chrome_options = Options()
        
        # Remove headless option as requested
        # chrome_options.add_argument('--headless')  # Commented out
        
        # Additional options to improve stability
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--window-size=1920,1080')
        chrome_options.add_argument('--ignore-certificate-errors')
        chrome_options.add_argument('--allow-running-insecure-content')
        
        # User agent to mimic browser
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        
        # Attempt to install and configure WebDriver
        service = Service(ChromeDriverManager().install())
        
        # Create WebDriver with extensive error handling
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # Additional configurations
        driver.set_page_load_timeout(50)  # 30 seconds page load timeout
        
        logging.info("WebDriver successfully initialized")
        return driver
    
    except Exception as e:
        logging.error(f"Failed to initialize WebDriver: {e}")
        raise

In [201]:
def scrape_straits_times_search_results(driver, url):
    """
    Scrape article details from Straits Times search results page
    
    Args:
        driver (webdriver): Selenium WebDriver
        url (str): URL of search results page
    
    Returns:
        list: List of dictionaries containing article details
    """
    try:
        logging.info(f"Navigating to URL: {url}")
        
        # Navigate to the URL
        driver.get(url)
        
        # Wait for search results
        WebDriverWait(driver, 50).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.queryly_item_row'))
        )
        # Find article containers
        article_containers = driver.find_elements(By.CSS_SELECTOR, '.queryly_item_row')
        logging.info(f"Found {len(article_containers)} articles")
        
        articles = []
        for container in article_containers:
            try:
                # Extract title
                try:
                    title_elem = container.find_element(By.CSS_SELECTOR, '.queryly_item_title')
                    title = title_elem.text.strip()
                except Exception as title_error:
                    logging.warning(f"Could not extract title: {title_error}")
                    title = 'N/A'
                
                # Extract URL
                try:
                    url_elem = container.find_element(By.CSS_SELECTOR, 'a')
                    article_url = url_elem.get_attribute('href')
                except Exception as url_error:
                    logging.warning(f"Could not extract URL: {url_error}")
                    article_url = 'N/A'
                
                # Extract description/summary
                try:
                    summary_elem = container.find_element(By.CSS_SELECTOR, '.queryly_item_description')
                    summary = summary_elem.text.strip()
                except Exception as summary_error:
                    logging.warning(f"Could not extract summary: {summary_error}")
                    summary = 'N/A'
                # Create article dictionary
                article = {
                    'Title': title,
                    'URL': article_url,
                    'Summary': summary
                }
                
                articles.append(article)
                logging.info(f"Extracted article: {article}")
            
            except Exception as container_error:
                logging.warning(f"Error processing article container: {container_error}")
        
        return articles
    
    except Exception as error:
        logging.error(f"Critical error scraping URL {url}: {error}")
        return []

In [202]:
def main():
    """
    Main function to process URLs and scrape search results
    """
    driver = None
    try:
        # Set up WebDriver
        driver = setup_webdriver()
        
        # Straits Times search URL
        url = "https://www.straitstimes.com/search?searchkey=%27Drug%20Trafficking%27"
        logging.info(f"Processing URL: {url}")
        
        # Scrape articles from the URL
        articles = scrape_straits_times_search_results(driver, url)
        
        # Save results to CSV
        if articles:
            output_df = pd.DataFrame(articles)
            output_filename = f'straits_times_drug_trafficking_articles_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            output_df.to_csv(output_filename, index=False, quoting=csv.QUOTE_ALL)
            logging.info(f"Scraped {len(articles)} articles. Saved to {output_filename}")
        else:
            logging.error("No articles were scraped")
    
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        # Always close the driver if it exists
        if driver:
            try:
                driver.quit()
                logging.info("WebDriver closed successfully")
            except Exception as quit_error:
                logging.error(f"Error closing WebDriver: {quit_error}")

In [203]:
if __name__ == '__main__':
    main()

# Pip Install Requirements:
# pip install pandas selenium webdriver-manager
#
# Notes:
# 1. Requires Google Chrome installation
# 2. Check straits_times_scraper_debug.log for detailed logs
# 3. Respect Straits Times' robots.txt and terms of service
# 4. Add delays and be considerate when scraping

2025-03-03 11:18:09,760 - INFO: ====== WebDriver manager ======
2025-03-03 11:18:10,776 - INFO: Get LATEST chromedriver version for google-chrome
2025-03-03 11:18:10,997 - INFO: Get LATEST chromedriver version for google-chrome
2025-03-03 11:18:11,251 - INFO: Driver [C:\Users\Bertrand Tan\.wdm\drivers\chromedriver\win64\133.0.6943.141\chromedriver-win32/chromedriver.exe] found in cache
2025-03-03 11:18:12,420 - INFO: WebDriver successfully initialized
2025-03-03 11:18:12,420 - INFO: Processing URL: https://www.straitstimes.com/search?searchkey=%27Drug%20Trafficking%27
2025-03-03 11:18:12,420 - INFO: Navigating to URL: https://www.straitstimes.com/search?searchkey=%27Drug%20Trafficking%27
2025-03-03 11:18:19,866 - INFO: Found 20 articles
2025-03-03 11:18:19,982 - INFO: Extracted article: {'Title': 'Accused Mexican cartel boss Caro Quintero pleads not guilty to US drug charges', 'URL': 'https://straitstimes.com/world/accused-mexican-cartel-boss-caro-quintero-pleads-not-guilty-to-us-drug-